# 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs

In [74]:
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/bnj/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache


In [11]:
#담을 데이터프레임 지정
scores= []
company= []

for i in range(1,18):
    url =f'http://www.38.co.kr/html/fund/index.htm?o=r1&page={i}' #homepage
    driver.get(url) 
    html=driver.page_source
    soup=BeautifulSoup(html)
    
    score1 = [] 
    score2 = []
    
    #17페이지까지 모든 종목들의 수요예측결과 크롤링
    for i in range(70):
        score1.append(soup.select('tr[bgcolor="#FFFFFF"] > td[align="center"]')[i].text)
        score2.append(soup.select('tr[bgcolor="#F8F8F8"] > td[align="center"]')[i].text)

    def list_chunk(lst, n):
        return [lst[i:i+n] for i in range(0, len(lst), n)]

    list_chunked1 = list_chunk(score1, 7)
    list_chunked2 = list_chunk(score2, 7)

    import itertools as it
    df= pd.DataFrame(list(it.chain(*zip(list_chunked1, list_chunked2))))
    scores.append(df)
    
    #종목명
    selection = soup.select('.menu')

    for sentence in selection:
        company.append([sentence.text.replace("\n",'').strip()],)

In [25]:
total_score_list=[]

data = pd.concat([scores[0],scores[1],scores[2],scores[3],scores[4],scores[5],scores[6],scores[7],scores[8],scores[9],
           scores[10],scores[11],scores[12],scores[13],scores[14],scores[15],scores[16]],axis=0).reset_index(drop=True)
data[7] = pd.DataFrame(results)

In [27]:
data.columns = ['예측일','공모희망가_원','공모가_원','공모금액_백만원','기관경쟁률','의무보유확약_퍼센트','주간사','종목명']
data = data[['종목명','예측일','공모희망가_원','공모가_원','공모금액_백만원','기관경쟁률','의무보유확약_퍼센트','주간사']]

In [6]:
data.to_csv("../project/data/공모주_기본정보.csv",index=False)

# 시초가

In [78]:
price=[]
title = []

for i in range(1,18):
    url =f'http://www.38.co.kr/html/fund/index.htm?o=nw&page={i}' #시초가 데이터 홈페이지
    driver.get(url)
    html=driver.page_source
    soup=BeautifulSoup(html)
    
    price1 = []
    price2 = []
    
    title1 = []
    title2 = []
    
    for i in range(1,21,2):
        price1.append(soup.select('tr[bgcolor="#FFFFFF"] > td[align="right"]')[3*i].text)
        price2.append(soup.select('tr[bgcolor="#F8F8F8"] > td[align="right"]')[3*i].text)


    for j in range(0,10):
        title1.append(soup.select('tr[bgcolor="#FFFFFF"] > td[style="border-left:#EFEFEF 1px solid;"]')[j].text)
        title2.append(soup.select('tr[bgcolor="#F8F8F8"] > td[style="border-left:#EFEFEF 1px solid;"]')[j].text)
    
    #종목명과 시초가 결합
    def list_chunk(lst, n):
        return [lst[i:i+n] for i in range(0, len(lst), n)]

    list_chunked1 = list_chunk(price1, 1)
    list_chunked2 = list_chunk(price2, 1)

    title_chunked1 = list_chunk(title1, 1)
    title_chunked2 = list_chunk(title2, 1)

    import itertools as it
    df1 = pd.DataFrame(list(it.chain(*zip(list_chunked1, list_chunked2))))
    df2 = pd.DataFrame(list(it.chain(*zip(title_chunked1,title_chunked2))))
    df2['target'] = df1
    title.append(df2)


In [80]:
title_price_list=[]

for i in range(0,17):
    title_price_list.append(title[i])
prices=pd.concat(title_price_list,ignore_index=True)
prices.to_csv("../project/data/공모주_시초가.csv",index=False)

In [82]:
df1 = pd.read_csv("./data/공모주_기본정보.csv")
df2 = pd.read_csv("./data/공모주_시초가.csv")
df2.columns=['종목명','시초가']
final_df = df1.merge(df2)
final_df.to_csv("./data/공모주_데이터.csv",index=False)

In [83]:
final_df

,종목명,예측일,공모희망가_원,공모가_원,공모금액_백만원,기관경쟁률,의무보유확약_퍼센트,주간사,시초가
0,위니아에이드,2022.06.09,"14,200~16,200","16,200","76,198",955:1,1.39%,신한금융투자,"14,600"
1,레이저쎌,2022.06.09,"12,000~14,000","16,000","19,200",1442.95:1,12.43%,삼성증권,"20,600"
2,KB스팩21호,2022.06.08,"2,000~2,000","2,000","15,000",NaN,-,KB증권,"2,100"
3,교보스팩12호,2022.06.08,"2,000~2,000","2,000","11,500",NaN,-,교보증권,"2,075"
4,보로노이,2022.06.08,"40,000~46,000","40,000","52,000",28.35 :1,-,"한국투자증권,미래에셋증권","36,000"
...,...,...,...,...,...,...,...,...,...
328,대모(구.대모엔지니어링),2019.07.09,"4,800~5,200","5,200","10,320",1103.1:1,15.60%,신한금융투자,"8,710"
329,에이스토리,2019.07.03,"11,600~14,300","14,300","21,692",614.25:1,3.56%,NH투자증권,"12,900"
330,플리토,2019.07.01,"19,000~23,000","26,000","27,996",1133:1,22.00%,한국투자증권,"31,600"
331,세틀뱅크,2019.06.27,"44,000~49,000","55,000","63,668",1122:1,14.80%,"한국투자증권,신한금융투자","49,500"
